# This script is used to automatically generate datasets formatted to be interpretable by DeepLabCut.

**WARNING :** Restrict the **Colony size** to a **maxmimum of 20 individuals**, otherwise the generation of datasets will fail, as the number of necessary columns would exceed the 64kb Metadata limit of the exported HDF5 file!

Ensure, that your **GPU** has sufficient **memory** for the chosen resolution! 

E.g. at 1024 px X 1024 px, training on an RTX 2080 Ti, set the **batchsize** to 4 in the **pose_cfg.yaml** file of your model.

In [1]:
import cv2
import pathlib
import json

import numpy as np
import pandas as pd
import matplotlib as plt

from os import listdir
from os.path import isfile, join

In [2]:
### REQUIRED ###

# define location of dataset and return all files
dataset_location = "E:/FARTS/FARTS_datasets/DLC/SYNTH/ORIG/stickmulti"
target_dir = "E:/FARTS/FARTS_datasets/DLC/SYNTH/MULTI-ANIMAL-HERO/labeled-data/multihero"
SCORER = "Fabi"

### OPTIONAL ###

# set True to show processing results for each image (disables parallel processing)
DEBUG = False

# we can optionally remove occluded points from the dataframe
EXCLUDE_OCCLUDED_KEYPOINTS = True

enforce_single_class = True # overwrites multiple classes and groups all instances as one

In [3]:
all_files = [f for f in listdir(dataset_location) if isfile(join(dataset_location, f))]
all_files.sort()

# next, sort files into images, depth maps, segmentation maps, data, and colony info
# we only need the location and name of the data files, as all passes follow the same naming convention
dataset_data = []
dataset_img = []
dataset_ID = []
dataset_depth = []
dataset_norm = []
dataset_colony = None

for file in all_files:
    loc = dataset_location + "/" + file
    file_info = file.split("_")
    
    if file_info[1] == "BatchData":
        dataset_colony = loc
        
    elif len(file_info) == 2:
        # images are available in various formats, but annotation data is always written as json files
        if file_info[-1].split(".")[-1] == "json":
            dataset_data.append(loc)
        else:
            dataset_img.append(loc)
            
    elif file_info[2].split(".")[0] == "ID":
        dataset_ID.append(loc)
    elif file_info[2].split(".")[0]  == "depth":
        dataset_depth.append(loc)
    elif file_info[2].split(".")[0]  == "norm":
        dataset_norm.append(loc)
        
print("Found",len(dataset_data),"samples...")

# next sort the colony info into its IDs to determine the colony size and individual scales
# Opening colony (BatchData) JSON file
colony_file = open(dataset_colony)
 
# returns JSON object as a dictionary
colony = json.load(colony_file)
colony_file.close()


""" !!! requires IDs, model names, scales !!! """


if not enforce_single_class:
    # get provided classes to create a dictionary of class IDs and class names
    subject_class_names = np.unique(np.array(colony["Subject Variations"]))
    subject_classes = {}
    for id,sbj in enumerate(subject_class_names):
        subject_classes[str(sbj)] = id
else:
    subject_class_names = np.array([0])
    subject_classes = {"insect" : 0}

print("\nA total of",len(subject_class_names),"unique classes have been found.")
print("The classes and respective class IDs are:\n",subject_classes,"\n")


print("Loaded colony file with seed", colony['Seed']) #,"and",len(colony['ID']),"individuals.")
    
multi_animal = False # focussing only one animal in each image
print("Generating Single-animal dataset! Containing",len(colony['Subject Variations']),"individuals")

Found 9999 samples...

A total of 1 unique classes have been found.
The classes and respective class IDs are:
 {'insect': 0} 

Loaded colony file with seed 1234567
Generating Single-animal dataset! Containing 10 individuals


Now that we have the cleaned colony info, we can start loading the data associated with each frame.
For simplicity we will simply make this a list of lists as the number of individuals.

We will therefore access "data" as [frame] [individual] [attribute], where attributes will include [ID,bbox_x_0,bbox_y_0,...]

To train a multi-animal DeepLabCut network, we mostly care about joint positions and less about bounding boxes.

As there may be animals for which we don't use all bones we can return a list of all labels and exclude the respective locations from the pose data. As all animals use the same convention, we can simply read in one example and remove the corresponding indices from all animals.

In [4]:
### REQUIRED ###
# specify which labels to ignore. By default, all keypoints are written into the dataset
# in this example we omit all keypoints relating to wings. Refer to the base_rig documentation for naming conventions
omit_labels = ['w_1_l', 'w_1_l_end', 'w_2_l', 'w_2_l_end', 'root']

# loading the first entry of first iteration file to retrieve skeleton info
exp_file = open(dataset_data[0])
exp_data = json.load(exp_file)
exp_file.close()

# for simplicity we'll assume that at this stage all subjects use the same armature and therefore report the same keypoints
first_entry_key = list(exp_data["iterationData"]["subject Data"][0].keys())[0]
labels = list(exp_data["iterationData"]["subject Data"][0][first_entry_key]["keypoints"].keys())

# show all used labels:
print("\nAll labels:",labels)

print("\nOmitting labels:", omit_labels)

# removing all occurences of omitted labels from the labels list to be used as keys below
labels = [x for x in labels if x not in omit_labels]

print("\nFinal labels:",labels)


All labels: ['root', 'b_t', 'b_a_1', 'b_a_2', 'b_a_3', 'b_a_4', 'b_a_5', 'l_1_co_r', 'l_1_tr_r', 'l_1_fe_r', 'l_1_ti_r', 'l_1_ta_r', 'l_1_pt_r', 'l_2_co_r', 'l_2_tr_r', 'l_2_fe_r', 'l_2_ti_r', 'l_2_ta_r', 'l_2_pt_r', 'l_3_co_r', 'l_3_tr_r', 'l_3_fe_r', 'l_3_ti_r', 'l_3_ta_r', 'l_3_pt_r', 'l_1_co_l', 'l_1_tr_l', 'l_1_fe_l', 'l_1_ti_l', 'l_1_ta_l', 'l_1_pt_l', 'l_2_co_l', 'l_2_tr_l', 'l_2_fe_l', 'l_2_ti_l', 'l_2_ta_l', 'l_2_pt_l', 'l_3_co_l', 'l_3_tr_l', 'l_3_fe_l', 'l_3_ti_l', 'l_3_ta_l', 'l_3_pt_l', 'b_h', 'ma_r', 'an_1_r', 'an_2_r', 'an_3_r', 'ma_l', 'an_1_l', 'an_2_l', 'an_3_l']

Omitting labels: ['w_1_l', 'w_1_l_end', 'w_2_l', 'w_2_l_end', 'root']

Final labels: ['b_t', 'b_a_1', 'b_a_2', 'b_a_3', 'b_a_4', 'b_a_5', 'l_1_co_r', 'l_1_tr_r', 'l_1_fe_r', 'l_1_ti_r', 'l_1_ta_r', 'l_1_pt_r', 'l_2_co_r', 'l_2_tr_r', 'l_2_fe_r', 'l_2_ti_r', 'l_2_ta_r', 'l_2_pt_r', 'l_3_co_r', 'l_3_tr_r', 'l_3_fe_r', 'l_3_ti_r', 'l_3_ta_r', 'l_3_pt_r', 'l_1_co_l', 'l_1_tr_l', 'l_1_fe_l', 'l_1_ti_l', 'l_1_ta_

Now that we have loaded data and colony info we can start plotting bounding boxes on top of their respective images

In [5]:
# transform between sRGB and linear colour space (optional)

def to_linear(srgb):
    linear = np.float32(srgb) / 255.0
    less = linear <= 0.04045
    linear[less] = linear[less] / 12.92
    linear[~less] = np.power((linear[~less] + 0.055) / 1.055, 2.4)
    return linear * 255.0

    
def from_linear(linear):
    srgb = linear.copy()
    less = linear <= 0.0031308
    srgb[less] = linear[less] * 12.92
    srgb[~less] = 1.055 * np.power(linear[~less], 1.0 / 2.4) - 0.055
    return srgb * 255.0

In [6]:
# let's create a big list to store all our dataset info and
# save it all to the desired .csv and .h5 files for DeepLabCut to read.

all_points = np.zeros((len(dataset_data) * len(colony['Subject Variations']), len(labels)*2))
#	- scorer   #(just one, the only scorer is the generator)
#	- - individuals
#	- - - bodyparts
#	- - - - coords

print("Number of loaded samples:",len(dataset_data))
print("Colony size:",len(colony['Subject Variations']))
print("body parts:",int(len(labels)),"\n")
print("Resulting in an array of shape:",all_points.shape)

output_file_names = ["" for i in range(len(dataset_data) * len(colony['Subject Variations']))]

Number of loaded samples: 9999
Colony size: 10
body parts: 51 

Resulting in an array of shape: (99990, 102)


In [7]:
# create unique colours for each ID
import numpy as np
import time

# alright. Let's take it from the top and fucking multi-thread this.
import threading
import queue
import sys
import os

def fix_bounding_boxes(coords,max_val = [1024,1024]):
    # fix bounding box coordinates so they do not reach beyond the image
    fixed_coords = []
    for c, coord in enumerate(coords):
        if c == 0 or c == 2:
            max_val_temp = max_val[0]
        else:
            max_val_temp = max_val[1]
            
        if coord >= max_val_temp:
            coord = max_val_temp
        elif coord <= 0:
            coord = 0
        
        fixed_coords.append(int(coord))
        
    return fixed_coords

def getThreads():
    """ Returns the number of available threads on a posix/win based system """
    if sys.platform == 'win32':
        return int(os.environ['NUMBER_OF_PROCESSORS'])
    else:
        return int(os.popen('grep -c cores /proc/cpuinfo').read())

class exportThread(threading.Thread):
    def __init__(self, threadID, name, q):
        threading.Thread.__init__(self)
        self.threadID = threadID
        self.name = name
        self.q = q

    def run(self):
        print("Starting " + self.name)
        process_detections(self.name, self.q)
        print("Exiting " + self.name)
        
def createThreadList(num_threads):
    threadNames = []
    for t in range(num_threads):
        threadNames.append("Thread_" + str(t))

    return threadNames

def process_detections(threadName, q):
    while not exitFlag_export:
        queueLock.acquire()
        if not workQueue_export.empty():
            
            data_input = q.get()
            i, data_loc, img, ID = data_input
            queueLock.release()
            
            display_img = cv2.imread(img)
            display_img_orig = display_img.copy()
            
            # compute visibility for each individual
            seg_img = cv2.imread(ID)
            seg_img_display = seg_img.copy()
            
            data_file = open(data_loc)
            # returns JSON object as a dictionary
            data = json.load(data_file)
            data_file.close()
            
            img_shape = display_img.shape
            
            # only add images that contain visibile individuals
            is_empty = True

            img_info = []
            
            # check if the size of the image and segmentation pass match
            if img_shape != seg_img.shape:
                print("Size mismatch of image and segmentation pass for sample",data_input[1].split("/")[-1],"!")
                incorrectly_formatted_images.append(i)
            else:
                for im, individual in enumerate(data["iterationData"]["subject Data"]):
                    ind_key = list(individual.keys())[0]
                    ind_ID = int(ind_key)
                    # WARNING ID numbering begins at 1
                    
                    img_name = img.split('/')[-1][:-4] + "_id_" + str(im) + "_synth" + ".png"
                    # write the file path to the all_points array
                    output_file_names[i * len(colony['Subject Variations']) + im] = "labeled-data/" + str(os.path.basename(target_dir)) + "/" + img_name

                    fontColor = (int(ID_colours[ind_ID,0]),
                                 int(ID_colours[ind_ID,1]),
                                 int(ID_colours[ind_ID,2]))
                    
                    bbox_orig = [individual[ind_key]["2DBounds"]["xmin"],
                                 individual[ind_key]["2DBounds"]["ymin"],
                                 individual[ind_key]["2DBounds"]["xmax"],
                                 individual[ind_key]["2DBounds"]["ymax"]]
                    
                    bbox = fix_bounding_boxes(bbox_orig, max_val=display_img.shape)
                    
                    # only process an individual if its bounding box width and height are not zero
                    if bbox[2] - bbox[0] == 0 or bbox[3] - bbox[1] == 0:
                        continue

                    try:
                        ID_mask = cv2.inRange(seg_img[bbox[1]:bbox[3],bbox[0]:bbox[2]], np.array([0, 0, ind_ID - 2]), np.array([0, 0, ind_ID + 2]))
                        indivual_occupancy = cv2.countNonZero(ID_mask)
                    except:
                        if len(threadList) == 1: 
                            print("Individual fully occluded:",ind_ID,"in",dataset_seg[i])
                        indivual_occupancy = 1

                    #indivual_occupancy = np.count_nonzero((seg_img == [0, 0, int((individual[0]/len(colony['ID']))*255)]).all(axis = 2)) + np.count_nonzero((seg_img == [0, 0, int((individual[0]/len(colony['ID']))*255 - 1)]).all(axis = 2)) + np.count_nonzero((seg_img == [0, 0, int((individual[0]/len(colony['ID']))*255 + 1)]).all(axis = 2))
                    bbox_area = abs((bbox[2] - bbox[0]) * (bbox[3] - bbox[1])) + 1
                    bbox_occupancy = indivual_occupancy / bbox_area
                    #print("Individual", individual[0], "with bounding box occupancy ",bbox_occupancy)

                    #cv2.putText(display_img, "ID: " + str(int(individual[0])), (bbox[0] + 10,bbox[3] - 10), font, fontScale, fontColor, lineType)
                    if bbox_occupancy > visibility_threshold:
                        # let's binarise the image and dilate it to make sure all points that visible are found
                        seg_bin = cv2.inRange(seg_img, np.array([0,0, ind_ID - 1]), np.array([0,0, ind_ID + 1]))
                        kernel = np.ones((5,5), np.uint8)
                        seg_bin_dilated = cv2.dilate(seg_bin,kernel,iterations = 2)
                        
                        for point in range(len(labels)):
                            # get rid of all invalid points first. Those should simply stay NaN in the array
                            if individual[ind_key]["keypoints"][labels[point]]["2DPos"]["x"] > img_shape[0] or individual[ind_key]["keypoints"][labels[point]]["2DPos"]["x"] < 0 or individual[ind_key]["keypoints"][labels[point]]["2DPos"]["y"] > img_shape[1] or individual[ind_key]["keypoints"][labels[point]]["2DPos"]["y"] < 0:
                                continue
                            else:
                                # now throw the coordinates to the correct location
                                out_row = i * len(colony['Subject Variations']) + im
                                out_column = point * 2
                                # exclude negative keypoints
                                if individual[ind_key]["keypoints"][labels[point]]["2DPos"]["x"] < 0.1 or individual[ind_key]["keypoints"][labels[point]]["2DPos"]["y"] < 0.1:
                                    individual[ind_key]["keypoints"][labels[point]]["2DPos"]["x"] = 0 # X
                                    individual[ind_key]["keypoints"][labels[point]]["2DPos"]["y"] = 0 # Y
                                # exlucde occluded keypoints by checking their visibility in the segmentation map   
                                if EXCLUDE_OCCLUDED_KEYPOINTS:
                                    x_temp = int(individual[ind_key]["keypoints"][labels[point]]["2DPos"]["x"])
                                    y_temp = int(individual[ind_key]["keypoints"][labels[point]]["2DPos"]["y"])           
                                    if seg_bin_dilated[y_temp,x_temp] == 0:  
                                        
                                        if DEBUG:
                                            display_img = cv2.circle(display_img, (x_temp,y_temp), radius=0, color=(0, 0, 255), thickness=2)
                                            cv2.imshow("missing points",display_img)
                                            cv2.waitKey(0)
                                        
                                        individual[ind_key]["keypoints"][labels[point]]["2DPos"]["x"] = 0 # X
                                        individual[ind_key]["keypoints"][labels[point]]["2DPos"]["y"] = 0 # Y
                                
                                if resize_resolution is not None:
                                    factor_X = resize_resolution / (bbox[2] - bbox[0])
                                    rescaled_X = round((individual[ind_key]["keypoints"][labels[point]]["2DPos"]["x"] - bbox[0]) * factor_X, 2)
                                    factor_Y = resize_resolution / (bbox[3] - bbox[1])
                                    rescaled_Y = round((individual[ind_key]["keypoints"][labels[point]]["2DPos"]["y"] - bbox[1]) * factor_Y, 2)
                                    all_points[out_row][out_column] = rescaled_X
                                    all_points[out_row][out_column + 1] = rescaled_Y
                                else:    
                                    all_points[out_row][out_column] = round(individual[ind_key]["keypoints"][labels[point]]["2DPos"]["x"] - bbox[0], 2) # X
                                    all_points[out_row][out_column + 1] = round(individual[ind_key]["keypoints"][labels[point]]["2DPos"]["y"] - bbox[1] ,2) # Y
                        
                        if resize_resolution is not None:
                            display_img_crop = display_img[bbox[1]:bbox[3],bbox[0]:bbox[2]]
                            resized_display_img = cv2.resize(display_img_crop, 
                                                             (resize_resolution, resize_resolution), 
                                                             interpolation = cv2.INTER_CUBIC)
                            cv2.imwrite(target_dir + "/" + img_name, resized_display_img)
                        else:
                            cv2.imwrite(target_dir + "/" + img_name, display_img[bbox[1]:bbox[3],bbox[0]:bbox[2]])
                    else:
                        incorrectly_formatted_images.append(i * len(colony['Subject Variations']) + im)
                        
            
        else:
            queueLock.release()
            
# setup as many threads as there are (virtual) CPU cores
exitFlag_export = 0
# only use a fourth of the number of CPUs for export as hugin and enfuse utilise multi core processing in part
threadList_export = createThreadList(getThreads())
print("Using", len(threadList_export), "threads for export...")
queueLock = threading.Lock()

# define paths to all images and set the maximum number of items in the queue equivalent to the number of images
workQueue_export = queue.Queue(len(dataset_img))
threads = []
threadID = 1

# keep track of all incorrectly formatted images to remove them after iterating over all entries
incorrectly_formatted_images = []

np.random.seed(seed=1)
ID_colours = np.random.randint(255, size=(255, 3))

font = cv2.FONT_HERSHEY_SIMPLEX
fontScale = 0.5
lineType = 2


# we can additionally plot the points in the data files to check joint locations
plot_joints = True

# remember to define an export folder when saving out your dataset
generate_dataset = True

# resize each sub window to a fixed resolution (set to None, if not desired)
resize_resolution = 500

# determine the proportion of a bounding box that needs to be filled before considering the visibility as too low
# WARNING: At the moment the ID shown in segmentation maps does not always correspond to the ID in the data file (off by 1)
visibility_threshold = 0.05

timer = time.time()

# Create new threads
for tName in threadList_export:
    thread = exportThread(threadID, tName, workQueue_export)
    thread.start()
    threads.append(thread)
    threadID += 1

# Fill the queue with samples
queueLock.acquire()
for i, (data, img, ID) in enumerate(zip(dataset_data , dataset_img, dataset_ID)):
    workQueue_export.put([i, data, img, ID])
queueLock.release()

# Wait for queue to empty
while not workQueue_export.empty():
    pass

# Notify threads it's time to exit
exitFlag_export = 1

# Wait for all threads to complete
for t in threads:
    t.join()
print("Exiting Main export Thread")

# close all windows if they were opened
cv2.destroyAllWindows()

# now, remove all incorrectly formatted imaged from the points and file list
incorrectly_formatted_images.sort()
print(incorrectly_formatted_images)

all_points = np.delete(all_points, incorrectly_formatted_images ,axis=0)
for r, rem_img in enumerate(incorrectly_formatted_images):
    del output_file_names[rem_img - r]

print("Total time elapsed:",time.time()-timer,"seconds")

Using 12 threads for export...
Starting Thread_0
Starting Thread_1
Starting Thread_2
Starting Thread_3Starting Thread_4

Starting Thread_5
Starting Thread_6
Starting Thread_7
Starting Thread_8
Starting Thread_9
Starting Thread_10
Starting Thread_11
Exiting Thread_8
Exiting Thread_1
Exiting Thread_11
Exiting Thread_6
Exiting Thread_4
Exiting Thread_5
Exiting Thread_2
Exiting Thread_7
Exiting Thread_0
Exiting Thread_3
Exiting Thread_10
Exiting Thread_9
Exiting Main export Thread
[7, 25, 42, 47, 57, 65, 69, 92, 107, 109, 114, 117, 119, 123, 125, 127, 129, 135, 147, 149, 152, 153, 157, 177, 203, 225, 275, 296, 315, 317, 345, 347, 356, 363, 368, 378, 379, 382, 393, 397, 398, 435, 449, 457, 462, 473, 474, 482, 483, 489, 537, 572, 577, 596, 597, 599, 642, 694, 727, 762, 833, 835, 837, 846, 862, 863, 887, 904, 918, 943, 948, 973, 1006, 1007, 1009, 1047, 1049, 1052, 1057, 1117, 1152, 1164, 1179, 1187, 1205, 1225, 1237, 1268, 1312, 1327, 1344, 1345, 1372, 1374, 1376, 1392, 1394, 1396, 1416, 1437

Now, dump it all into one **DLC-conform pandas (.h5)** file

In [8]:
#example_DLC_df = pd.read_hdf("I:/FARTS/DeepLabCut-Multi-Animal/multi_ant_test_label-Fabi-2021-07-23/labeled-data/multi_animal_1080p/CollectedData_Fabi.h5")
#print(example_DLC_df.columns.get_level_values(2))
#example_DLC_df

In [9]:
# next create the required hierarchy
scorer = [SCORER for i in range(len(all_points[0]))]
individuals = []

for ind in range(len(colony['Subject Variations'])):
    ### UPDATE ONCE ALL COLONY INFO IS INCLUDED ###
    individual = ["id_" + str(ind) +"_num_" + str(ind) for i in range(int((len(labels))*2))]
    individuals.extend(individual)
    
bodyparts_filtered = [i for j in labels for i in [j]*2]
bodyparts = []

for i in range(len(colony['Subject Variations'])):
    bodyparts.extend(bodyparts_filtered)

coords = []
for i in range(int(len(all_points[0])/2)):
    coords.extend(["x","y"])

Now that all elemts for the **Multi-Index** hierachy are defined, we can combine them into the **final dataframe**  

In [10]:
categories = [scorer, bodyparts, coords]
categories_tuples = list(zip(*categories))
columns = pd.MultiIndex.from_tuples(categories_tuples, names=["scorer",
                                                           "bodyparts",
                                                           "coords"])

final_dataframe = pd.DataFrame(all_points, index = output_file_names, columns=columns)
# convert all zeros, negative values, and those esxceeding the cropped image to NaN
final_dataframe[final_dataframe < 0] = 0 # set negative values to 0
if resize_resolution is not None:
    final_dataframe[final_dataframe > resize_resolution] = 0
final_dataframe = final_dataframe.replace(0, np.nan)
final_dataframe = final_dataframe.dropna(thresh=2)

In [11]:
final_dataframe

scorer                                                Fabi                  \
bodyparts                                              b_t           b_a_1   
coords                                                   x       y       x   
labeled-data/multihero/stickmulti_00000_id_0_sy...  234.30  348.47  234.91   
labeled-data/multihero/stickmulti_00000_id_1_sy...  154.03  233.73  332.23   
labeled-data/multihero/stickmulti_00000_id_2_sy...  377.03  260.04  300.16   
labeled-data/multihero/stickmulti_00000_id_4_sy...   95.29  149.34  255.01   
labeled-data/multihero/stickmulti_00000_id_5_sy...  259.85   25.40  337.55   
...                                                    ...     ...     ...   
labeled-data/multihero/stickmulti_09999_id_3_sy...  161.39  331.18  210.30   
labeled-data/multihero/stickmulti_09999_id_5_sy...  120.37  299.32  287.13   
labeled-data/multihero/stickmulti_09999_id_6_sy...  365.10  220.29     NaN   
labeled-data/multihero/stickmulti_09999_id_7_sy...     NaN     NaN     NaN   
labeled-data/multihero/stickmulti_09999_id_8_sy...  329.94  306.07  177.15   

scorer                                                                      \
bodyparts                                                    b_a_2           
coords                                                   y       x       y   
labeled-data/multihero/stickmulti_00000_id_0_sy...  161.79  225.68  106.57   
labeled-data/multihero/stickmulti_00000_id_1_sy...  378.20  386.48  407.72   
labeled-data/multihero/stickmulti_00000_id_2_sy...   95.80  288.42   54.37   
labeled-data/multihero/stickmulti_00000_id_4_sy...  186.40  314.97  186.41   
labeled-data/multihero/stickmulti_00000_id_5_sy...  353.56  382.87  423.89   
...                                                    ...     ...     ...   
labeled-data/multihero/stickmulti_09999_id_3_sy...  280.17  214.93  271.50   
labeled-data/multihero/stickmulti_09999_id_5_sy...  499.59     NaN     NaN   
labeled-data/multihero/stickmulti_09999_id_6_sy...     NaN     NaN     NaN   
labeled-data/multihero/stickmulti_09999_id_7_sy...     NaN     NaN     NaN   
labeled-data/multihero/stickmulti_09999_id_8_sy...  372.18  126.24  386.84   

scorer                                                                      \
bodyparts                                            b_a_3           b_a_4   
coords                                                   x       y       x   
labeled-data/multihero/stickmulti_00000_id_0_sy...  217.56   51.23  215.23   
labeled-data/multihero/stickmulti_00000_id_1_sy...  437.32  441.44  476.18   
labeled-data/multihero/stickmulti_00000_id_2_sy...  277.37   13.92  281.03   
labeled-data/multihero/stickmulti_00000_id_4_sy...  366.28  176.57  390.85   
labeled-data/multihero/stickmulti_00000_id_5_sy...  428.81  486.27  473.93   
...                                                    ...     ...     ...   
labeled-data/multihero/stickmulti_09999_id_3_sy...  207.30  272.81  189.78   
labeled-data/multihero/stickmulti_09999_id_5_sy...     NaN     NaN     NaN   
labeled-data/multihero/stickmulti_09999_id_6_sy...     NaN     NaN     NaN   
labeled-data/multihero/stickmulti_09999_id_7_sy...     NaN     NaN     NaN   
labeled-data/multihero/stickmulti_09999_id_8_sy...   77.49  414.46   34.03   

scorer                                                      ...          \
bodyparts                                                   ...  an_3_r   
coords                                                   y  ...       x   
labeled-data/multihero/stickmulti_00000_id_0_sy...   19.81  ...  167.60   
labeled-data/multihero/stickmulti_00000_id_1_sy...  475.24  ...    3.69   
labeled-data/multihero/stickmulti_00000_id_2_sy...    0.85  ...  362.49   
labeled-data/multihero/stickmulti_00000_id_4_sy...  153.67  ...  129.27   
labeled-data/multihero/stickmulti_00000_id_5_sy...  498.90  ...     NaN   
...                                                    ...  ...     ...   
labeled-data/multihero/stickmulti_09999_id_3_s

In [12]:
final_dataframe.to_csv(os.path.join(target_dir, "CollectedData_FARTS.csv"))

# IF the function below fails, this is likely due to exceeding the number of columns supported by HDF5 files!
# Restrict the number of simulated animals to < 20 if the goal is to train a DLC network

final_dataframe.to_hdf(
    os.path.join(target_dir, "CollectedData_" + "FARTS" + ".h5"),
    "df_with_missing",
    format="table",
    mode="w")
    